In [1]:
@decorate
def target():
    print('running target()')

NameError: name 'decorate' is not defined

## Ex 7-1

In [2]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target')

target()

running inner()


In [3]:
target

<function __main__.deco.<locals>.inner()>

# 7.2 When python runs decorator

## Decorator runs right after defining a decorated function

## Ex 7-2

In [2]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == "__main__":
    main()

running register(<function f1 at 0x0000014C6CEF31F8>)
running register(<function f2 at 0x0000014C6CED8CA8>)
running main()
registry -> [<function f1 at 0x0000014C6CEF31F8>, <function f2 at 0x0000014C6CED8CA8>]
running f1()
running f2()
running f3()


## 7.3 Stratagy Pattern improved by Decorator

In [5]:
promos = list()

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    
    return discount

@promotion
def large_order(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.card}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

# 7.4 Variable Range Rule

## Ex 7-4

In [6]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [10]:
b = 6
def f1(a):
    print(a)
    print(b)

f1(3)

3
6


In [11]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [12]:
b = 6

def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [13]:
from dis import dis
dis(f1)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [14]:
dis(f2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## 7.5 Closure

In [15]:
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [16]:
avg = Averager()

In [17]:
avg(10)

10.0

In [18]:
avg(11)

10.5

In [19]:
avg(12)

11.0

In [22]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [23]:
avg = make_averager()

In [24]:
avg(10)

10.0

In [25]:
avg(11)

10.5

In [26]:
avg(12)

11.0

## series is a "free variable"

In [27]:
avg.__code__.co_varnames

('new_value', 'total')

In [28]:
avg.__code__.co_freevars

('series',)

In [29]:
avg.__code__.co_freevars

('series',)

In [30]:
avg.__closure__

(<cell at 0x0000014C6CEDC468: list object at 0x0000014C6CF8A788>,)

In [31]:
avg.__closure__[0].cell_contents

[10, 11, 12]

## 7.6 nonlocal

## Ex 7-13

In [33]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [34]:
avg = make_averager()

In [35]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

## Ex 7-14

In [36]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

# 7.7 Simple decorator

## 7-15 a decorator that prints executed time for a function

In [3]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    
    return clocked

## Ex 7-16 clock decorator

In [4]:
import time

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12282656s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000032s] factorial(1) -> 1
[0.00001283s] factorial(2) -> 2
[0.00002117s] factorial(3) -> 6
[0.00002919s] factorial(4) -> 24
[0.00003689s] factorial(5) -> 120
[0.00004748s] factorial(6) -> 720
6! = 720


In [5]:
factorial.__name__

'clocked'

## Ex 7-17 improved clock decorator

In [ ]:
import time
import functools

def clock(func):
    @functools.wrap(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    return clocked